## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


In [1]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* **4**
* 7


In [2]:
! ls -l ../data

total 97780
-rw-rw-rw- 1 codespace codespace  1427002 Mar 20  2023 green_tripdata_2023-01.parquet
-rw-rw-rw- 1 codespace codespace  1533740 May  1  2023 green_tripdata_2023-02.parquet
-rw-rw-rw- 1 codespace codespace  1730999 May 19  2023 green_tripdata_2023-03.parquet
-rw-rw-rw- 1 codespace codespace 47673370 Mar 20  2023 yellow_tripdata_2023-01.parquet
-rw-rw-rw- 1 codespace codespace 47748012 May  1  2023 yellow_tripdata_2023-02.parquet


In [3]:
%run preprocess_data.py --raw_data_path ../data --dest_path ../output


In [4]:
! ls -l ../output

total 7016
-rw-rw-rw- 1 codespace codespace  131004 May 17 09:02 dv.pkl
-rw-rw-rw- 1 codespace codespace 2458698 May 17 09:02 test.pkl
-rw-rw-rw- 1 codespace codespace 2374518 May 17 09:02 train.pkl
-rw-rw-rw- 1 codespace codespace 2215824 May 17 09:02 val.pkl


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* **2**
* 4
* 8
* 10



`mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns`

In [13]:
%run train.py --data_path ../output --experiment_name "nyc-taxi-experiment"

2025/05/17 09:26:46 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Validation RMSE: 29.4975


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* **`default-artifact-root`**
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* **5.335**
* 5.818
* 6.336

In [15]:
%run hpo.py --data_path ../output

🏃 View run rf-hypo-train at: http://127.0.0.1:5000/#/experiments/2/runs/2d0ad0abc4f74ed09b8700beccfe7aa3                   

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                

🏃 View run rf-hypo-train at: http://127.0.0.1:5000/#/experiments/2/runs/fcfa1e112d7542c2b4f34a82959d435f                   

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                

🏃 View run rf-hypo-train at: http://127.0.0.1:5000/#/experiments/2/runs/7c0da83ff38e49429a4a98c85dd6c8e8                   

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                

🏃 View run rf-hypo-train at: http://127.0.0.1:5000/#/experiments/2/runs/478211e503a8419c96e48ba7d9c9fccd                   

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                                                                



## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* **5.567**
* 6.061
* 6.568

In [30]:
%run register_model.py --data_path ../output

🏃 View run unleashed-kit-70 at: http://127.0.0.1:5000/#/experiments/3/runs/67e3cc2edf214c5e8479edd92d7ba9f8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run sedate-auk-585 at: http://127.0.0.1:5000/#/experiments/3/runs/08f4aa724ede4748be881a861b6fb5d1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run resilient-fawn-191 at: http://127.0.0.1:5000/#/experiments/3/runs/8b4811ea88cb474f8c4f419e97130dcf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run resilient-ram-499 at: http://127.0.0.1:5000/#/experiments/3/runs/801a042cbe754823a118d1e1c2fec49d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


Successfully registered model 'random-forest-hyperopt'.
2025/05/17 10:45:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-hyperopt, version 1


🏃 View run gentle-lynx-162 at: http://127.0.0.1:5000/#/experiments/3/runs/b2808f8586d54cb082580144827bd045
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


Created version '1' of model 'random-forest-hyperopt'.
